<a href="https://colab.research.google.com/github/swaprash/NLP-Specialization/blob/master/My_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [83]:
import pandas as pd
import emoji
import spacy
pd.options.display.max_colwidth = 100
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers

In [31]:
sp = spacy.load('en_core_web_sm')

In [32]:
data=pd.read_csv('/content/Usecase3_Dataset.csv')

In [33]:
data.shape

(14640, 3)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  object
 1   airline            14640 non-null  object
 2   text               14640 non-null  object
dtypes: object(3)
memory usage: 343.2+ KB


In [35]:
data.describe()

,airline_sentiment,airline,text
count,14640,14640,14640
unique,3,6,14427
top,negative,United,@united thanks
freq,9178,3822,6


In [36]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials to the experience... tacky.
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I need to take another trip!
3,negative,Virgin America,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &..."
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing about it


In [37]:
data.isna().sum()

airline_sentiment    0
airline              0
text                 0
dtype: int64

In [38]:
#Removal of hastags/account
data['text'] = data['text'].str.replace('(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)', " ")

In [39]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,What said.
1,positive,Virgin America,plus you've added commercials to the experience... tacky.
2,neutral,Virgin America,I didn't today... Must mean I need to take another trip!
3,negative,Virgin America,"it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they hav..."
4,negative,Virgin America,and it's a really big bad thing about it


In [40]:
#Removal of Punctuation
data['text'] = data['text'].str.replace('[\.\,\!\?\:\;\-\=]', " ")

In [41]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,What said
1,positive,Virgin America,plus you've added commercials to the experience tacky
2,neutral,Virgin America,I didn't today Must mean I need to take another trip
3,negative,Virgin America,"it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp they hav..."
4,negative,Virgin America,and it's a really big bad thing about it


In [42]:
data.text

0                                                                                               What   said 
1                                                  plus you've added commercials to the experience    tacky 
2                                                   I didn't today    Must mean I need to take another trip 
3          it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp  they hav...
4                                                                   and it's a really big bad thing about it
                                                        ...                                                 
14635                                                     thank you we got on a different flight to Chicago 
14636      leaving over 20 minutes Late Flight  No warnings or communication until we were 15 minutes Lat...
14637                                                                    Please bring American Airlines to  
14638      you have

In [43]:
#Lower case
data['text'] =data['text'].str.lower()

In [44]:
data.text

0                                                                                               what   said 
1                                                  plus you've added commercials to the experience    tacky 
2                                                   i didn't today    must mean i need to take another trip 
3          it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp  they hav...
4                                                                   and it's a really big bad thing about it
                                                        ...                                                 
14635                                                     thank you we got on a different flight to chicago 
14636      leaving over 20 minutes late flight  no warnings or communication until we were 15 minutes lat...
14637                                                                    please bring american airlines to  
14638      you have

In [45]:
data['text']=[emoji.demojize(data['text'].loc[i]) for i in range(len(data['text']))]

In [46]:
data.text.loc[18]

'i :red_heart_selector: flying    :smiling_face_selector::thumbs_up:'

In [47]:
 data['text'] = data['text'].str.replace(":"," ")

In [48]:
data.text.loc[18]

'i  red_heart_selector  flying     smiling_face_selector  thumbs_up '

In [49]:
#data['text'] = data['text'].str.split()

In [50]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,what said
1,positive,Virgin America,plus you've added commercials to the experience tacky
2,neutral,Virgin America,i didn't today must mean i need to take another trip
3,negative,Virgin America,"it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp they hav..."
4,negative,Virgin America,and it's a really big bad thing about it


In [51]:
contraction= {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }



In [52]:
type(contraction)

dict

In [53]:
#data['text']=data['text'].replace(contraction,regex=True,inplace=True)

In [54]:
data.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,what said
1,positive,Virgin America,plus you've added commercials to the experience tacky
2,neutral,Virgin America,i didn't today must mean i need to take another trip
3,negative,Virgin America,"it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp they hav..."
4,negative,Virgin America,and it's a really big bad thing about it


In [55]:
data['text']=data['text'].replace(contraction,regex=True)

In [56]:
data.text

0                                                                                               what   said 
1                                                plus you have added commercials to the experience    tacky 
2                                                  i did not today    must mean i need to take another trip 
3          it is really aggressive to blast obnoxious "entertainment" in your guests' faces &amp  they ha...
4                                                                  and it is a really big bad thing about it
                                                        ...                                                 
14635                                                     thank you we got on a different flight to chicago 
14636      leaving over 20 minutes late flight  no warnings or communication until we were 15 minutes lat...
14637                                                                    please bring american airlines to  
14638      you have

In [149]:
df=data[['text','airline_sentiment']]

In [150]:
df.head()

,text,airline_sentiment
0,what said,neutral
1,plus you have added commercials to the experience tacky,positive
2,i did not today must mean i need to take another trip,neutral
3,"it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp they ha...",negative
4,and it is a really big bad thing about it,negative


In [130]:
df.shape

(14640, 2)

In [151]:
attribute = df.text
target = df.airline_sentiment

In [152]:
attribute_train, attribute_test, target_train, target_test = train_test_split(attribute, target, test_size = 0.1, random_state = 225)

print('attribute_train :', len(attribute_train))
print('attribute_test  :', len(attribute_test))
print('target_train :', len(target_train))
print('target_test  :', len(target_test))

attribute_train : 13176
attribute_test  : 1464
target_train : 13176
target_test  : 1464


In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tvec = TfidfVectorizer()
clf2 = LogisticRegression()

In [154]:
from sklearn.pipeline import Pipeline

model = Pipeline([('vectorizer',tvec)
                 ,('classifier',clf2)])

model.fit(attribute_train, target_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, inter

In [161]:
example_text = ["you service was not good enough"]
example_result = model.predict(example_text)

print(example_result)

['negative']


# Test

In [156]:
from sklearn.metrics import confusion_matrix

verdict = model.predict(attribute_test)

confusion_matrix(verdict, target_test)

array([[846, 141,  60],
       [ 30, 181,  31],
       [  6,  16, 153]])

In [157]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Accuracy : ", accuracy_score(verdict, target_test))
print("Precision : ", precision_score(verdict, target_test, average = 'weighted'))
print("Recall : ", recall_score(verdict, target_test, average = 'weighted'))

Accuracy :  0.8060109289617486
Precision :  0.849447150721419
Recall :  0.8060109289617486
